# 2023 Day 12

https://adventofcode.com/2023/day/12

https://adventofcode.com/2023/day/12/input

In [1]:
from functools import lru_cache

import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

In [2]:
inp = open('input-12.txt').read().strip().split('\n')
inp

['#?#???????#?.? 3,1,2,2',
 '.??.##.?#????.??#? 2,2,1,2,4',
 '.?.??#?##??#????.. 8,1',
 '??????????#.. 6,1',
 '.#.##????.??.? 1,6,1',
 '?#??#???.??.??? 3,1,1,1,1',
 '?#???????#? 2,1,5',
 '##?#??#?????.??? 5,1,4,2',
 '????.?#??.# 2,1,1',
 '???????##?????##???? 1,1,11,3',
 '.####??.??#??????#? 6,6,1',
 '??#..##?...?#. 2,3,2',
 '..??##?#?#???#??.??. 12,1',
 '??.?#???#? 1,3,1',
 '??.#??????????#.??? 4,1,1,3,1',
 '???##??.?##? 5,1,2',
 '???..???..?. 1,1,1,1',
 '.?????#?#??? 1,1,1,1',
 '##???????????# 5,3,2',
 '??..????###?#?.? 1,8',
 '????????#???.##?? 1,2,1,2,3',
 '????##.?#????.? 1,4,2,1,1',
 '##???#.##?.#????? 2,2,3,2,3',
 '?????.????? 2,3',
 '?#?#??.##???.?#?. 3,5,2',
 '?#?..?##???#?????? 1,12',
 '.??#?????????.?.#.? 11,1,1,1',
 '???????????#??????. 7,1,1,3,1',
 '??????????#?????## 1,1,2,1,1,6',
 '.##?.##?????????? 2,4,1,1',
 '??#?##????? 5,1',
 '??#????????????? 5,2,2',
 '?#.#??##??#???#????? 1,2,13',
 '#??.????#.?? 1,2,2,1',
 '#.#?#???????.???#. 1,4,1,1,1,4',
 '????#??..?.. 2,1,1,1',


In [3]:
test1 = """???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1
""".strip().split('\n')
test1

['???.### 1,1,3',
 '.??..??...?##. 1,1,3',
 '?#?#?#?#?#?#?#? 1,3,1,6',
 '????.#...#... 4,1,1',
 '????.######..#####. 1,6,5',
 '?###???????? 3,2,1']

In [4]:
def parse_line(line, factor=1):
    row, counts = line.split()
    counts = tuple(map(int, counts.split(',')))
    return '?'.join(factor * [row]), factor * counts

In [5]:
def find_allowed(s, count):
    s = '.' + s + '.'
    allowed = []
    for i in range(1, len(s) - count):
        if s[i - 1] == '#' or s[i + count] == '#':
            continue
        substr = s[i : i + count]
        assert len(substr) == count
        if '.' in set(substr):
            continue
        # print(substr)
        allowed.append(i - 1)
    return allowed

In [6]:
@lru_cache(maxsize=1000000)
def _count_options(s, counts, level, verbose):
    if not counts:
        return 0 if '#' in s else 1
    
    def pr(*a, **kw):
        if verbose:
            print(*a, flush=True, **kw)
    
    i_count = np.argmax(counts)
    count = counts[i_count]
    allowed = find_allowed(s, count)
    counts_left = counts[:i_count]
    counts_right = counts[i_count + 1:]
    out = 0
    space = level * '  '
    # print(space + f'{s=}  {counts=}  {i_count=}  {count=}  {allowed=} {level=}')
    pr(space + f'{counts_left=}  {counts_right=}')
    for i_match in allowed:
        # pr(s[:i_match-1], counts_left, level+1)
        kw = dict(level=level + 1, verbose=verbose)
        s_left = s[:i_match - 1] if i_match else ''
        s_right = s[i_match+count+1:] if i_match < len(s) - 1 else ''
        pr(space + f'<<< {i_match=} {s_left} {s_right}')
        
        n_left = count_options(s_left, counts_left, **kw)
        n_right = count_options(s_right, counts_right, **kw)
        
        # n_left = count_options(s_left, counts_left, **kw) if counts_left else 1
        # n_right = count_options(s_right, counts_right, **kw) if counts_right else 1
        out += n_left * n_right
        pr(space + f'>>> {i_match=} {n_left=} {n_right=} {out=}')
    return out


def count_options(s, counts, level=0, verbose=False):
    return _count_options(s, counts, level, verbose)

## Part 1

In [7]:
[
    count_options(*parse_line(line))
    for line in test1
]

[1, 4, 1, 1, 4, 10]

In [8]:
sum([
    count_options(*parse_line(line))
    for line in test1
])

21

In [9]:
sum([
    count_options(*parse_line(line))
    for line in inp
])

7007

## Part 2

In [10]:
[
    count_options(*parse_line(line, factor=5))
    for line in test1
]

[1, 16384, 1, 16, 2500, 506250]

In [11]:
sum([
    count_options(*parse_line(line, factor=5))
    for line in test1
])

525152

In [12]:
sum([
    count_options(*parse_line(line, factor=5))
    for line in tqdm(inp)
])

100%|████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:12<00:00, 13.86it/s]


3476169006222